In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import re
import glob
from tqdm import tqdm
import platform

In [2]:
def check_os():
    os = platform.system()

    if os == 'Darwin':
        return "MacOS"
    elif os == 'Linux':
        return "Linux"
    else:
        return "Unknown OS"

In [3]:
operating_system = check_os()
    
if operating_system == "MacOS":
    root_path = "/Users/johnny/Projects/"
elif operating_system == "Linux":
    root_path = "/home/johnny/Projects/"

In [4]:
data = os.getcwd()
ANNOTATIONS_PATH = root_path + "datasets/harbor-synthetic/annotations"
LTD_DATASET_PATH = root_path + "datasets/harbor-synthetic/LTD_Dataset/LTD_Dataset/Image_Dataset_25fps"

In [5]:
def get_top_level_folders(base_path):
    folder_list = []

    for entry in os.listdir(base_path):
        entry_path = os.path.join(base_path, entry)
        if os.path.isdir(entry_path):
            entry_path = entry_path.replace(base_path + "/", "")
            folder_list.append(entry_path)

    # Sort the list 20200514, 20200515, ...
    folder_list.sort(key=lambda f: int(re.sub("\D", "", f)))
    return folder_list


def get_all_files_folder(path, type_file="jpg"):
    files = glob.glob(path + f"/*.{type_file}")
    sorted_files = sorted(files, key=lambda f: int(re.sub("\D", "", f)))
    return sorted_files

In [6]:
annotation_folders = get_top_level_folders(ANNOTATIONS_PATH)

In [7]:
annotation_folders

['20200514',
 '20200515',
 '20200516',
 '20200517',
 '20200518',
 '20200519',
 '20200520',
 '20200521',
 '20200522',
 '20200523',
 '20200524',
 '20200525',
 '20200526',
 '20200527',
 '20200528',
 '20200529',
 '20200530',
 '20200531',
 '20200601',
 '20200602',
 '20200603',
 '20200604',
 '20200605',
 '20200606',
 '20200607',
 '20200608',
 '20200609',
 '20200610',
 '20200611',
 '20200612',
 '20200613',
 '20200614',
 '20200615',
 '20200616',
 '20200617',
 '20200618',
 '20200624',
 '20200625',
 '20200626',
 '20200627',
 '20200628',
 '20200629',
 '20200630',
 '20200701',
 '20200706',
 '20200707',
 '20200708',
 '20200709',
 '20200710',
 '20200711',
 '20200712',
 '20200713',
 '20200810',
 '20200811',
 '20200812',
 '20200813',
 '20200814',
 '20200815',
 '20200816',
 '20200817',
 '20200818',
 '20200819',
 '20200820',
 '20200821',
 '20200822',
 '20200823',
 '20200824',
 '20200825',
 '20200826',
 '20200827',
 '20200828',
 '20200829',
 '20200830',
 '20200831',
 '20200901',
 '20210108',
 '20210109',

In [8]:
def associate_frame_with_annotation(annotations_folders, annotations_path, frames_path):
    my_dict = dict()
    date_folders = annotations_folders

    for date_folder in tqdm(date_folders):
        path_subfolder = os.path.join(annotations_path, date_folder)
        date = os.path.basename(path_subfolder)
        date_frame_path = os.path.join(frames_path, date)

        clips_annotations = get_top_level_folders(path_subfolder)

        for clip_annotation in clips_annotations:
            path_clip = os.path.join(path_subfolder, clip_annotation)
            path_frame = os.path.join(date_frame_path, clip_annotation)
            annotations_txt = get_all_files_folder(path_clip, "txt")
            frames_jpg = get_all_files_folder(path_frame, "jpg")

            for idx, annotation_txt in enumerate(annotations_txt):
                frame_jpg = frames_jpg[idx]
                num_frame = os.path.basename(frame_jpg).split(".")[0].split("_")[1]

                if date_folder not in my_dict:
                    my_dict[date_folder] = {}
                if clip_annotation not in my_dict[date_folder]:
                    my_dict[date_folder][clip_annotation] = {}
                if num_frame not in my_dict[date_folder][clip_annotation]:
                    my_dict[date_folder][clip_annotation][num_frame] = {}
                
                "removes up to /../../"
                frame_jpg = frame_jpg.replace(data + "/../../../", "")
                annotation_txt = annotation_txt.replace(data + "/../../../", "")
                    
                # create temp dict structure {datefolder: {clip_annotation: {frame:num_frame, path_frame: frame_jpg, path_annotation: annotation_txt }}}}}
                temp_dict = {
                    "path_frame": frame_jpg if path_frame else "",
                    "path_annotation": annotation_txt if annotation_txt else "",
                }
                # add new element to dict 
                my_dict[date_folder][clip_annotation][num_frame] = temp_dict

    # Concatenate all DataFrames in the list
    return my_dict

In [9]:
frame_annotation_map = associate_frame_with_annotation(
    annotation_folders, ANNOTATIONS_PATH, LTD_DATASET_PATH
)

  0%|          | 0/188 [00:00<?, ?it/s]

100%|██████████| 188/188 [00:21<00:00,  8.88it/s]


In [10]:
# save frame_annotation_map to .json
import json
with open('frame_annotation_map.json', 'w') as fp:
    json.dump(frame_annotation_map, fp)

In [11]:
# read folder and subfolder from annotations folder and replace txt
# 00000009 human 77 109 86 119 1 by 00000009 human 77 109 86 119 1 0
"""
import os
def add_loitering_txt(root_folder):
    for root, dirs, files in os.walk(root_folder):
        for filename in files:
            if filename.endswith(".txt"):
                file_path = os.path.join(root, filename)
                new_filename = filename.replace(".txt", "_loitering.txt")
                new_file_path = os.path.join(root, new_filename)
                os.rename(file_path, new_file_path)

# Example usage:
folder_path = "/Users/johnny/Projects/datasets/harbor-synthetic/annotations"
add_loitering_txt(folder_path)
"""


'\nimport os\ndef add_loitering_txt(root_folder):\n    for root, dirs, files in os.walk(root_folder):\n        for filename in files:\n            if filename.endswith(".txt"):\n                file_path = os.path.join(root, filename)\n                new_filename = filename.replace(".txt", "_loitering.txt")\n                new_file_path = os.path.join(root, new_filename)\n                os.rename(file_path, new_file_path)\n\n# Example usage:\nfolder_path = "/Users/johnny/Projects/datasets/harbor-synthetic/annotations"\nadd_loitering_txt(folder_path)\n'